## scoring
### for scoring districting plans

In [1]:
from gerrychain import Graph, Partition, Election
from gerrytools.scoring import *
import pandas as pd
import geopandas as gpd

In [2]:
%load_ext autoreload
%autoreload 2

All of our scores are functions that take a GerryChain `Partition` and produce either a numerical (plan-wide) score or a mapping from district or election IDs to numeric scores. For our examples, we will use a 2020 Maryland VTD shapefile to build our underlying dual graph, since the shapefile has demographic and electoral information that our scores will rely on.

In [3]:
%%time
graph = Graph.from_file("data/MD_vtd20/")

CPU times: user 12 s, sys: 113 ms, total: 12.1 s
Wall time: 12.1 s


In [4]:
elections = ["PRES12", "SEN12", "GOV14", "AG14", "COMP14", 
             "PRES16", "SEN16", "GOV18", "SEN18", "AG18", "COMP18"]

# use our list of elections ablve to create `Election` updaters for each contest
# Ex: in our shapefile, the column `PRES12R` refers to the votes Mitt 
# Romney (R) received in the 2012 Presidential general election
updaters = {}
for e in elections:
    updaters[e] = Election(e, {"Dem": e+"D", "Rep": e+"R"})

The `demographic_updaters()` function returns a dictionary of `Tally` updaters that track the number of people of a given demographic group. You can pass as a list with as many demographic groups as you wish (example below):

In [5]:
demographic_updaters(["TOTPOP20", "VAP20"])

{'TOTPOP20': <gerrychain.updaters.tally.Tally at 0x7f9e3a7eae00>,
 'VAP20': <gerrychain.updaters.tally.Tally at 0x7f9e3a7eb800>}

In [6]:
# add updaters that track total population, total voting age population, 
# and Black and Hispanic voting age population
updaters.update(demographic_updaters(["TOTPOP20", "VAP20", "BVAP20", "HVAP20", "WVAP20"]))

# create the partition on which we'll generate scores
# since `MD_CD_example.csv` is a CSV with `GEOID20` -> district assignment,
# we need to replace the `GEOID20`s with integer node labels to match the graph's nodes.
geoid_to_assignment = pd.read_csv("data/MD_CD_example.csv", header=None).set_index(0).to_dict()[1]
assignment = {n: geoid_to_assignment[graph.nodes[n]["GEOID20"]] for n in graph.nodes}
partition = Partition(graph, assignment, updaters)

### partisan scores
All our partisan scores require at least a list of elections (we'll use our `elections` list defined above). Some of them additionally require the user to specify a POV party (in our case, either `Dem` or `Rep`). All of these partisan scores return a dictionary that maps election names to the score for that election; it is up to the user to aggregate (often by summing or averaging) the scores across every election. For a simple example, let's use the score function that returns the number of Democratic seats won in each election.

In [7]:
seats(elections, "Dem")

Score(name='Dem_seats', apply=functools.partial(<function _seats at 0x7f9e45344f40>, election_cols=['PRES12', 'SEN12', 'GOV14', 'AG14', 'COMP14', 'PRES16', 'SEN16', 'GOV18', 'SEN18', 'AG18', 'COMP18'], party='Dem', mean=False), dissolved=False)

Note that the output of `seats(elections, "Dem")` is of type `Score`, which functions like a Python `namedtuple`: for any object `x` of type `Score`, `x.name` returns the name of the score, and `x.apply` returns a function that takes a `Partition` as input and returns the score. See below:

In [8]:
seats(elections, "Dem").name

'Dem_seats'

In [9]:
seats(elections, "Dem").apply(partition)

{'PRES12': 6,
 'SEN12': 6,
 'GOV14': 4,
 'AG14': 6,
 'COMP14': 6,
 'PRES16': 6,
 'SEN16': 6,
 'GOV18': 4,
 'SEN18': 6,
 'AG18': 6,
 'COMP18': 8}

Note that we can easily find the number of Republican seats like so:

In [10]:
seats(elections, "Rep").apply(partition)

{'PRES12': 2,
 'SEN12': 2,
 'GOV14': 4,
 'AG14': 2,
 'COMP14': 2,
 'PRES16': 2,
 'SEN16': 2,
 'GOV18': 4,
 'SEN18': 2,
 'AG18': 2,
 'COMP18': 0}

Moreover, we can pass `mean=True` to return the average of the score over all elections, rather than a dictionary:

In [11]:
seats(elections, "Rep", mean=True).apply(partition)

2.1818181818181817

Some partisan scores (`mean_median`, `efficiency_gap`, `partisan_bias`, `partisan_gini`) do not require the user to specify the POV party in the call. This is not because there isn't a POV party, but because these functions call GerryChain functions that automatically set the POV party to be the **first** party listed in the updater for that election. Since we always list `Dem` first in this notebook, this means `Dem` will be the POV party for these scores— but this is something you should keep in mind when setting up your updaters and your partition.

In [12]:
# Positive values denote an advantage for the POV party
efficiency_gap(elections).apply(partition)

{'PRES12': -0.027366954931038075,
 'SEN12': -0.1112428189930485,
 'GOV14': -0.016952521996415275,
 'AG14': 0.0664089504401374,
 'COMP14': -0.03643474212627552,
 'PRES16': -0.04564932242915228,
 'SEN16': -0.02799189191120642,
 'GOV18': 0.09144998629410322,
 'SEN18': -0.12475998763996132,
 'AG18': -0.06082242557828398,
 'COMP18': 0.05664447794898745}

If you know you want to use a lot of scores, it can be helpful to make a list of the scores of interest, like so:

In [13]:
partisan_scores = [
    seats(elections, "Dem"),
    seats(elections, "Rep"),
    # signed_proportionality(elections, "Dem", mean=True),
    # absolute_proportionality(elections, "Dem", mean=True),
    efficiency_gap(elections, mean=True),
    mean_median(elections),
    partisan_bias(elections),
    partisan_gini(elections),
    # Note that `eguia` takes several more arguments — see the documentation for more details
    eguia(elections, "Dem", graph, updaters, "COUNTYFP20", "TOTPOP20"),
]

Now, we can make use of the `summarize()` function to evaluate all the scores on this partition:

In [14]:
partisan_dictionary = summarize(partition, partisan_scores)
partisan_dictionary["mean_median"]

{'PRES12': 0.02205704780736839,
 'SEN12': 0.04184519796735442,
 'GOV14': 0.0128224074264629,
 'AG14': 0.03372274606966308,
 'COMP14': 0.026622499095666607,
 'PRES16': 0.03478025159124121,
 'SEN16': 0.03829214902714728,
 'GOV18': 0.0195942524690087,
 'SEN18': 0.037782714199074086,
 'AG18': 0.03906798945053658,
 'COMP18': 0.036168324606223434}

In [15]:
partisan_dictionary["mean_efficiency_gap"]

-0.02151975008383212

### demographic scores

Our demographic scores return a dictionary that maps districts to demographic information, either population counts or shares.

In [16]:
# `demographic_tallies()` takes a list of the demographics you'd like to tally
tally_scores = demographic_tallies(["TOTPOP20", "BVAP20", "HVAP20"])
tally_dictionary = summarize(partition, tally_scores)
tally_dictionary

{'TOTPOP20': {1: 771992,
  7: 772346,
  8: 772421,
  6: 771907,
  3: 773001,
  4: 772893,
  5: 771418,
  2: 771246},
 'BVAP20': {1: 50513,
  7: 186256,
  8: 84454,
  6: 285475,
  3: 106681,
  4: 258794,
  5: 334253,
  2: 82315},
 'HVAP20': {1: 40466,
  7: 36221,
  8: 27363,
  6: 44099,
  3: 45359,
  4: 144187,
  5: 43594,
  2: 110973}}

In [17]:
# `demographic_shares()` takes a dictionary where each key is a total demographic column
# that will be used as the denominator in the share (usually either `TOTPOP20` or `VAP20`)
# and each value is a list of demographics on which you'd like to compute shares
share_scores = demographic_shares({"VAP20": ["BVAP20", "HVAP20"]})
share_dictionary = summarize(partition, share_scores)
share_dictionary

{'BVAP20_share': {1: 0.08427654278144459,
  7: 0.3075109503392005,
  8: 0.1389347687326854,
  6: 0.463149987751003,
  3: 0.18038569170027308,
  4: 0.4331758821894971,
  5: 0.5577436821598711,
  2: 0.13770530746350554},
 'HVAP20_share': {1: 0.06751399798455716,
  7: 0.05980131717762746,
  8: 0.045014707140366,
  6: 0.07154549893977225,
  3: 0.07669701811787184,
  4: 0.2413438137099663,
  5: 0.07274213867961521,
  2: 0.1856474650446164}}

#### Two things to note:

Both `demographic_tallies()` and `demographic_shares()` return _lists_ of `Score`s (one for each demographic of interest), so if we want to just score one demographic, we'd have to index into the list in order to call `.function()`:

In [18]:
demographic_tallies(["BVAP20"])[0].apply(partition)

{1: 50513,
 7: 186256,
 8: 84454,
 6: 285475,
 3: 106681,
 4: 258794,
 5: 334253,
 2: 82315}

Moreover, you can only use these scores on demographic columns that have already been tracked as `Tally` updaters when we instantiated our partition. If you try a new column (say, `WVAP20`) things won't work!

In [19]:
demographic_tallies(["HVAP20"])

[Score(name='HVAP20', apply=functools.partial(<function _tally_pop at 0x7f9e85e41e40>, pop_col='HVAP20'), dissolved=False)]

In [20]:
demographic_tallies(["WVAP20"])[0].apply(partition)

{1: 457669,
 7: 320218,
 8: 458845,
 6: 234283,
 3: 348325,
 4: 127814,
 5: 178346,
 2: 275860}

Our last demographic updater is `gingles_districts()`, which takes in a dictionary of the same type as `demographic_tallies()` as well as a `threshold` between 0 and 1. Just like the other two demographic scores it returns a list of `Score`s, but here the `Score`s represent the number of districts where the demographic group's share is above the `threshold`. (When the threshold is 0.5 — the default — these districts are called _Gingles' Districts_.

In [21]:
gingles_scores = gingles_districts({"VAP20": ["BVAP20", "HVAP20"]}, threshold=0.5)
gingles_dictionary = summarize(partition, gingles_scores)
gingles_dictionary

{'BVAP20_gingles_districts': 1, 'HVAP20_gingles_districts': 0}

### compactness scores

We can count the number of county _splits_ (the number of counties that are assigned to more than one district) as well the number of county _pieces_ (the sum of the number of unique $($_county_, _district_$)$ pairs over every split county).

By passing a column name to the `pop_col` keyword argument, you can specify whether you just want splits and pieces that impact population. 

In [22]:
# if we had a column in our data defining municipal boundaries, we could
# similarly county municipal splits/pieces
county_scores = [
    splits("COUNTYFP20"),
    pieces("COUNTYFP20"),
]
county_dictionary = summarize(partition, county_scores)
county_dictionary

{'COUNTYFP20_splits': 5, 'COUNTYFP20_pieces': 12}

##  compactness scores

We can also score for compactness in a variety of different ways. 

The functions `reock`, `polsby_popper`, `schwartzberg`, `convex_hull`, and `pop_poplygon`, get each of these scores for each district in a plan. Unlike the other scoring functions, each of these takes a GeoDataFrame and a crs as arguments. Most of these require the use of a pre-dissolved GeoDataFrame by plan district. This is so the geometries can be used for calculations. 

Below, we go through the call to each of these functions, which all take similar arguments.

In [23]:
md_example = pd.read_csv("data/MD_CD_example.csv", header=None).rename(columns={0:"GEOID20", 1: "assignment"})
vtd_gdf = gpd.read_file("data/MD_vtd20")
dissolved_gdf = gpd.GeoDataFrame(md_example.merge(vtd_gdf, on = "GEOID20"), geometry="geometry").dissolve(by="assignment", aggfunc="sum").reset_index()

In [29]:
compactness_scores = [reock()]
                    #   polsby_popper(dissolved_gdf, dissolved_gdf.crs), 
                    #   schwartzberg(dissolved_gdf, dissolved_gdf.crs),
                    #   convex_hull(dissolved_gdf, dissolved_gdf.crs, assignment_col="assignment"), 
                    #   pop_polygon(vtd_gdf, dissolved_gdf, dissolved_gdf.crs, pop_col="TOTPOP20", assignment_col="assignment")]
compactness_dictionary = summarize(compactness_scores, partition)
# compactness_dictionary

KeyError: 0

# summary

We can string together all of our score lists and use them to fully summarize our partition:

In [ ]:
all_scores = partisan_scores + tally_scores + share_scores + gingles_scores + county_scores + compactness_scores
summary_dictionary = summarize(partition, all_scores)
summary_dictionary

## misc

Other miscellaneous scores we can use:

In [ ]:
# max_deviation() gives us the maximum deviation from ideal district population, either as a count or as a percent
max_deviation("TOTPOP20").apply(partition)

In [ ]:
max_deviation("TOTPOP20", pct=True).apply(partition)